In [8]:
import pandas as pd
import numpy as np
import glob


In [52]:
pgr_metal = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Metal',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_pet = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Petreo',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_madera = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Maderas',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_plat = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Plasticos',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_nonrec = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='No Reciclable',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_papel = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Papeles y Cartones',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_pelig = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Peligrosos',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_suelo = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Suelo',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))
pgr_vidrio = pd.DataFrame(pd.read_excel('Planilla_residuosA.xlsx',sheet_name='Vidrio',usecols=['FECHA','ORIGEN','UNIDAD','CANTIDAD']))




#['FECHA', 'TICKET N', 'DESCRIPCION DE MATERIAL', 'ORIGEN', 'DESTINO',
   #    'UNIDAD', 'CANTIDAD', 'RETIRADO POR']


In [53]:
pgr_total = pd.concat([pgr_metal,pgr_pet,pgr_madera,pgr_plat,pgr_nonrec,pgr_papel,pgr_pelig,pgr_suelo,pgr_vidrio])

nombre = pgr_total['ORIGEN'].unique()

total = pgr_total['CANTIDAD'].sum()

print(f'El total de residuos es de la {nombre} es de {total} kg')


El total de residuos es de la ['NWT ' 'NWT' nan 'ANULADO'] es de 788229.5 kg


C:\Users\HP\AppData\Local\Temp\ipykernel_8208\3486699767.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pgr_total = pd.concat([pgr_metal,pgr_pet,pgr_madera,pgr_plat,pgr_nonrec,pgr_papel,pgr_pelig,pgr_suelo,pgr_vidrio])


In [62]:
pgr_total.set_index('FECHA', inplace=True)

In [64]:
import matplotlib.pyplot as plt